In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from transformers import AutoModel, AutoTokenizer
import pandas as pd

import math
import tqdm

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [88]:
test_num = 1

In [89]:
if test_num == 1:
    df = pd.read_csv('../data/hackathon_test_for_user.csv', encoding='cp949')
elif test_num == 2:
    df = pd.read_excel('../data2/test_data.xlsx')

In [54]:
question = df['Q_number']

In [55]:
dq = pd.read_excel('../data/Question.xlsx', index_col=0)

In [56]:
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
model = AutoModel.from_pretrained("klue/roberta-large")

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

In [7]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        data = {k:v[idx] for k,v in self.data.items()}
        return data

In [8]:
answer_map = {'그렇다':0, '그렇자':0,
              '중립':1, '중랍':1, '중간':1, '보통':1, '중립/모르겠다': 1,
              '어렵다':2, '아니다':2, '<아니다':2, '아니요':2, '아니오':2}
def combine(dfx, filenum):
    list = []
    for i in range(len(dfx)):
        if filenum == 1:
            div = dfx.Answer[i].index('>')+1
            short_answer = dfx.Answer[i][:div].replace(' ', '')[1:-1]
            long_answer = dfx.Answer[i][div:]
        elif filenum == 2:
            short_answer = dfx.Short_Answer[i]
            long_answer = dfx.Long_Answer[i]
        answer_code = answer_map[short_answer]
        q = dfx.Q_number[i]
        prefix = {0:dq.Positive[q], 1:'', 2:dq.Negative[q]}[answer_code]
        if type(prefix) == float:
            prefix = ''
        list.append(prefix+' '+long_answer)
    return list

In [9]:
comb_final = combine(df, test_num)

In [17]:
comb_final

['이미 있던 친구들과만 지내요. 친구를 만들 상황에 새로운 친구를 만듭니다. 의도적으로나 꼭 주기적으로 새로운 친구를 만들지는 않습니다.\n',
 '자유 시간 중 상당 부분을 다양한 관심사를 탐구하는 데 할애해요. 일상이 무료한 걸 못 참는 편. 요즘은 리본으로 뜨개질해서 가방 만드는 걸 배우고 있는 중입니다.\n',
 '다른 사람이 울고 있는 모습을 보아도 무심해요. 다른 사람이 울고 있는 모습을 본다고 해도 눈물이 나지는 않습니다. 그 심정은 이해가 가지만 굳이 울 일인가라는 생각을 많이 했던 것 같습니다.',
 '일이 잘못될 때를 대비해 여러 대비책을 세우는 편이에요. 진행하고 있는 일이 계획단계일 때 이미 대비책을 함께 세우는 편. 잘못된 순간 바로 이어서 진행해야 시간 낭비 없이 마감기한을 맞출 수 있기 때문입니다.',
 '언제나 평정심을 유지하는 편이에요. 최근 프로젝트 진행 중 문제가 발생, 마감기한이 얼마 남지 않았고, 주변 사람들도 함께 동요되어 일을 그르칠 수 있는 상황에 평정심을 갖고 해결하여 일을 잘 마무리할 수 있었습니다.\n',
 ' 파티나 행사를 갔는데, 아는 사람을 만난다면 일단 대화를 하다가 새로운 사람과 대화를 할 것 같습니다.',
 '여러 프로젝트를 동시에 진행하기 좋아하는 편이에요. 보통은 동시에 여러 가지 일을 하는 것을 즐기는 편입니다. 시간 낭비 없이 일을 효율적으로 처리할 수 있기 때문입니다.',
 '이성적인 편이에요. 감상적인 편은 아닌 것 같습니다. 분위기나 감정보다는 숫자와 그래프를 믿는 편입니다.',
 '일정이나 목록으로 계획을 세우는 일을 좋아해요. 주로 다이어리를 사용합니다. 자기 전에 오늘 해야 할 일을 다 마무리했는지 확인하고, 다음날 해야 할 to do list를 작성합니다.',
 '작은 실수로도 제 능력이나 지식을 의심하곤 해요. 친구들과 여행을 갈 때 계획을 세우는 편인데, 계획대로 일정을 소화하지 못할 때, 친구들의 체력까지 감안하지 못하고 계획을 세운 나에게 화가 났습니다.',
 '관심이 가는

In [18]:
tensor = tokenizer(comb_final, max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
ds = MyMapDataset(tensor)
dl = DataLoader(ds, batch_size=64, shuffle=False)

C:\Program Files\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2364: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [19]:
tokenizer.decode(tensor.input_ids[0])

'[CLS] 이미 있던 친구들과만 지내요. 친구를 만들 상황에 새로운 친구를 만듭니다. 의도적으로나 꼭 주기적으로 새로운 친구를 만들지는 않습니다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [20]:
def forward(model, dl):
    pooled = []
    hidden = []
    model.eval()
    for data in tqdm.tqdm(dl):
        with torch.no_grad():
            data = data.to(device)
            output = model(data, output_hidden_states=True).cpu()
        p, h = output.pooler_output, output.last_hidden_state
        pooled.append(p)
        hidden.append(h[:,0,:]) # only [CLS] token embedding
    return torch.cat(pooled), torch.cat(hidden)

In [21]:
result = forward(model, dl)[1]

  0%|          | 0/113 [00:53<?, ?it/s]


KeyboardInterrupt: 

In [24]:
result.shape

torch.Size([2880, 1024])

In [25]:
torch.save(result, 'roberta/test_final.pt')

In [57]:
result = torch.load('roberta/test_final.pt')

In [58]:
#question_result = torch.load('question_embed.pt')
#question_list = question_result[1].tolist()
gender_map = {0:[1,0], 1:[0,1], 'male':[1,0], 'female':[0,1]}
age_map = {20:[1,0,0], 30:[0,1,0], 40:[0,0,1]}
def attach(result, df, filenum):
    result = result.tolist()
    for i in range(len(result)):
        result[i].extend(gender_map[df.Gender[i]])
        result[i].extend(age_map[df.Age[i]])
    return torch.tensor(result)

In [59]:
#question_result = torch.load('question_embed.pt')
#question_list = question_result[1].tolist()
gender_map = {0:0, 1:3}
age_map = {20:0, 30:1, 40:2}
def attach(result, df, filenum):
    result = result.tolist()
    for i in range(len(result)):
        extend_l = [0 for i in range(6)]
        if filenum == 1:
            extend_l[gender_map[df.Gender[i]]+age_map[df.Age[i]]] = 1
        result[i].extend(extend_l)
    return torch.tensor(result)

In [60]:
result_real = attach(result, df, test_num).float()

In [61]:
result_real[-1].tolist()[-200:]

[-0.1143970787525177,
 0.5610578060150146,
 0.28671103715896606,
 -0.554839015007019,
 0.1419786512851715,
 0.18929368257522583,
 0.030677707865834236,
 -0.04977761581540108,
 -0.2511250078678131,
 -0.03931815177202225,
 -0.23971694707870483,
 0.0375211276113987,
 0.3967617452144623,
 -0.20724140107631683,
 -8.155599594116211,
 -0.012381382286548615,
 -0.15572084486484528,
 0.05183251574635506,
 0.17504648864269257,
 0.4668087363243103,
 0.011348461732268333,
 -0.10707388073205948,
 -0.019874755293130875,
 0.24722455441951752,
 0.0919409766793251,
 -0.2067074030637741,
 0.3479980528354645,
 0.04065736010670662,
 -0.1110677570104599,
 0.0608692392706871,
 0.5687393546104431,
 -0.18509970605373383,
 0.1718561202287674,
 -0.030678557232022285,
 0.04099216312170029,
 -0.2420298457145691,
 -0.053009532392024994,
 -0.1398146003484726,
 -0.09111045300960541,
 0.04082953929901123,
 -0.19659830629825592,
 0.5492411851882935,
 0.01019801665097475,
 -0.2088734358549118,
 -0.23167812824249268,
 -0

In [62]:
def main(path):
    output_list = [[] for i in range(61)]
    model = torch.load(path, map_location='cpu')
    model.eval()
    with torch.no_grad():
        output = model(result_real)
        for i in range(len(output)):
            output_list[question[i]].append((i, output[i]))
    print(output[:10])
    def normalize(l):
        def sigmoid_inv(x):
            return -math.log(1/x-1)
        sigmoid = nn.Sigmoid()
        inv = torch.tensor(list(map(sigmoid_inv, l.tolist())))
        left = -torch.max(inv)
        right = -torch.min(inv)
        error = 1
        while error > 1e-6:
            mid = (left+right)/2
            shift = sigmoid(inv+mid)
            mean = torch.mean(shift)
            if mean < 0.5:
                left = mid
            else:
                right = mid
            error = torch.abs(mean-0.5)
        return shift
    def aggregate(l):
        return torch.exp(torch.mean(torch.log(l))).item()
    if test_num == 1:
        predict = torch.zeros((len(result_real), 4))
        for l in output_list:
            if l:
                tensors = torch.transpose(torch.stack([t[1] for t in l]), 0, 1)
                for i in range(len(tensors)):
                    tensors[i] = normalize(tensors[i])
                for i in range(len(l)):
                    predict[l[i][0]] = tensors[:,i]
    elif test_num == 2:
        predict = torch.zeros((120, 4))
        output = torch.reshape(output, (120, 60, 4))
        for i in range(120):
            for j in range(4):
                predict[i][j] = aggregate(output[i,:,j])
    return predict

In [69]:
answer = main('ckpt/ckpt85/epoch_249.pt')

tensor([[8.9648e-01, 1.4160e-01, 6.9413e-01, 4.7877e-01],
        [4.2512e-01, 1.5287e-01, 9.9777e-01, 8.4158e-01],
        [2.0128e-01, 9.4404e-01, 8.9639e-01, 9.2140e-02],
        [9.6077e-01, 6.0882e-01, 8.6509e-01, 3.3772e-01],
        [9.6107e-01, 6.6507e-01, 4.6633e-01, 6.5248e-01],
        [1.6138e-02, 6.3095e-01, 7.6344e-01, 3.0486e-01],
        [1.3296e-02, 6.0895e-01, 9.7282e-01, 6.2645e-01],
        [7.9647e-02, 8.8871e-01, 8.4034e-01, 9.1363e-01],
        [8.4792e-04, 9.8601e-01, 9.9843e-01, 6.9704e-01],
        [3.9283e-01, 4.9622e-01, 8.5307e-01, 6.5399e-01]])


In [70]:
torch.mean(answer[:,2])

tensor(0.5000)

In [109]:
test = pd.DataFrame({
    'I/E': answer[:,0].tolist(),
    'S/N': answer[:,1].tolist(),
    'T/F': answer[:,2].tolist(),
    'J/P': answer[:,3].tolist()
})
if test_num == 2:
    test['User_ID'] = df.User_ID.unique()
test.index += 1

In [110]:
test

,I/E,S/N,T/F,J/P,User_ID
1,0,0,0,0,1
2,0,1,0,1,2
3,1,0,1,1,3
4,1,0,1,1,4
5,1,0,0,0,5
...,...,...,...,...,...
116,0,1,0,0,175
117,0,1,0,0,178
118,1,0,0,1,179
119,1,0,0,1,191


In [111]:
if test_num == 1:
    test.to_csv('submissions/test17-5.csv', index_label='idx')
elif test_num == 2:
    test.to_csv('submissions2/testr3.csv', columns=['User_ID', 'I/E', 'S/N', 'T/F', 'J/P'], index=False)